In [1]:

%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import NearestNeighbors
import scipy.stats as stats
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier

import glob
import os
import scipy as sp
import scipy.stats


from sklearn.covariance import EllipticEnvelope

import scipy.stats as stats




In [2]:
plot_2d = 0
std_range = 3
def generate_data():
    FIRST_ROW_NUM = 0

    # dir_file = "/Users/tester/Documents/Vz_datamining/UDP_BiDir_Throughput/20180713-170739/unicast_unidirectional_throughput/BidirResults.csv"
    dir_file = '/Users/tester/Documents/Vz_datamining/'
    start_num = 0
    num_samples = 1000
    outfile_path = dir_file + 'output.csv'
    i = 0
    for path, dirs, files in os.walk(dir_file):
        for d in dirs:
            for filename in glob.iglob(os.path.join(path, d, '*BidirResults.csv')):
                print(filename)
                with open(filename, 'rt') as infile, open(outfile_path, 'wt') as outfile:
                    outfile.writelines(row for row_num, row in enumerate(infile, FIRST_ROW_NUM)
                                        if (((row_num < start_num + num_samples) and (row_num > start_num) ) or (row_num == 0)))
                df_Bidir = pd.read_csv(open(outfile_path,'r',errors='ignore'),skip_blank_lines=True, error_bad_lines=False,low_memory=False)
                df_Bidir = df_Bidir.dropna()
                Bidir_1d = np.reshape(df_Bidir.values, (1,np.shape(df_Bidir.values)[0]*np.shape(df_Bidir.values)[1]))
                if i == 0:
                    Bidir_all = Bidir_1d
                else:
                    Bidir_all = np.append(Bidir_all,Bidir_1d, axis = 0)
                i = i + 1
    return Bidir_all

def index_to_name(index_item):
    test_item = ["Frame Size", "Trial", "Trial Duration (sec)", "ILOAD Total pkts/sec", "Throughput Up pkts/sec", "Throughput Up bits/sec", "Throughput Down pkts/sec", "Throughput Down bits/sec", "Offered Load Up bits/sec", "Offered Load Down bits/sec"]
    frame_size = ["64","88","128","256","512","1024","1280","1518"]
    frame_size_txt = frame_size[int(index_item/len(test_item))]
    test_item_txt = test_item[index_item%len(test_item)]
    test_name = "frame size "+frame_size_txt + " " + test_item_txt
    return test_name
    

def mean_confidence_interval(data, std_range):
    a = 1.0*np.array(data)
    n = len(a)
    m = np.mean(a)
    std = np.std(a)
    return [[m, m-std*std_range, m+std*std_range]]

def detetct_outlier_single_item(Bidir_all,std_range = 3):
    for i in range(np.shape(Bidir_all)[1]):
        confident_interval_id = mean_confidence_interval(Bidir_all[:,i],std_range)
        if i == 0:
            confident_interval = confident_interval_id
        else:
            confident_interval = np.append(confident_interval,confident_interval_id, axis = 0)
    # print(confident_interval)

    outlier_matrix = np.zeros(np.shape(Bidir_all))
    round_matrix = np.zeros(np.shape(Bidir_all)[0])
    item_matrix = np.zeros(np.shape(Bidir_all)[1])


    num_outlier = 0
    for i in range(np.shape(Bidir_all)[0]):
        for j in range(np.shape(Bidir_all)[1]):
            if (Bidir_all[i][j] > confident_interval[j][2]) or (Bidir_all[i][j] < confident_interval[j][1]):
    #             print("sample",i, j, "is outlier", "value is ",Bidir_all[i][j], "(", confident_interval[j][1], "~",confident_interval[j][2] )
                num_outlier = num_outlier + 1
                outlier_matrix[i][j] = 1
                round_matrix[i] = 1
                item_matrix[j] = 1
    item_outlier = np.where(item_matrix == 1)[0]
    return round_matrix,item_outlier,num_outlier


def plot_outlier_2d(Bidir_all, envelope,threshold,pred):
        x1s = np.linspace(np.min(Bidir_all[:, 0])-5, np.max(Bidir_all[:, 0])+5, 15)
        x2s = np.linspace(np.min(Bidir_all[:, 1])-5, np.max(Bidir_all[:, 1])+5, 15)
        x1grid, x2grid = np.meshgrid(x1s, x2s)

        Xgrid = np.column_stack((x1grid.ravel(), x2grid.ravel()))  # Feature matrix containing all grid points
        dens = envelope.decision_function(Xgrid)
        densgrid = dens.reshape(x1grid.shape)  # Reshape the vector of densities back onto the "grid"
        fig, ax = plt.subplots()
        ax.contourf(x1grid, x2grid, densgrid, cmap=plt.cm.Blues_r, levels=np.linspace(dens.min(), threshold, 7))
        ax.scatter(Bidir_all[:, 0], Bidir_all[:, 1], s=8, color="g")
        # Pot circles around the predicted outliers
        ax.scatter(Bidir_all[pred == -1, 0], Bidir_all[pred == -1, 1],  
                   facecolors="none", edgecolors="red", s=80, label="predicted outliers")
        ax.legend(loc="lower right")
        
def detect_outlier_multiple(Bidir_all,array_of_interests, plot_indicator):
    # Bidir_all = np.dstack((Bidir_all[:,4],Bidir_all[:,5],Bidir_all[:,6],Bidir_all[:,7],Bidir_all[:,8],Bidir_all[:,9],Bidir_all[:,10],Bidir_all[:,11],Bidir_all[:,12],Bidir_all[:,13],Bidir_all[:,16],Bidir_all[:,17],Bidir_all[:,18],Bidir_all[:,19],Bidir_all[:,20],Bidir_all[:,21],Bidir_all[:,22],Bidir_all[:,25],Bidir_all[:,26],Bidir_all[:,27],Bidir_all[:,28],Bidir_all[:,29],Bidir_all[:,30],Bidir_all[:,31],Bidir_all[:,32],Bidir_all[:,33],Bidir_all[:,34],Bidir_all[:,35],Bidir_all[:,36],Bidir_all[:,37],Bidir_all[:,38],Bidir_all[:,39],Bidir_all[:,40],Bidir_all[:,41],Bidir_all[:,42],Bidir_all[:,43],Bidir_all[:,44],Bidir_all[:,45],Bidir_all[:,46],Bidir_all[:,47],Bidir_all[:,48],Bidir_all[:,49],Bidir_all[:,50],Bidir_all[:,51],Bidir_all[:,52],Bidir_all[:,53],Bidir_all[:,54],Bidir_all[:,55],Bidir_all[:,56],Bidir_all[:,57],Bidir_all[:,58],Bidir_all[:,59],Bidir_all[:,60],Bidir_all[:,61],Bidir_all[:,62],Bidir_all[:,63],Bidir_all[:,64],Bidir_all[:,65],Bidir_all[:,66],Bidir_all[:,67],Bidir_all[:,68],Bidir_all[:,69],Bidir_all[:,70],Bidir_all[:,71],Bidir_all[:,72],Bidir_all[:,73],Bidir_all[:,74],Bidir_all[:,75],Bidir_all[:,76],Bidir_all[:,77],Bidir_all[:,78],Bidir_all[:,79]))[0]
    num_pairs = np.shape(array_of_interests)[0]
    my_xticks = ["64","88","128","256","512","1024","1280","1518"]
    for i in range(num_pairs):
        Bidir_all_new = np.dstack((Bidir_all[:,array_of_interests[i][0]], Bidir_all[:,array_of_interests[i][1]]))[0] 

        # 15 is the key 
        # print(np.shape(Bidir_all))

        outlier_frac = .02
        envelope = EllipticEnvelope(contamination = outlier_frac)
        envelope.fit(Bidir_all_new)


        # Use the densites as the "z" values in a contour plot on the grid
        pred = envelope.predict(Bidir_all_new)
        
        threshold = stats.scoreatpercentile(envelope.decision_function(Bidir_all_new), 100*outlier_frac)
        Bidir_prediction_matrix = envelope.predict(Bidir_all_new)
        outlier_indices = np.where(Bidir_prediction_matrix == -1)[0]
        print("Frame size",my_xticks[i],"detected outliers is",outlier_indices )
        if plot_indicator:
            plot_outlier_2d(Bidir_all_new, envelope,threshold,pred)
    return outlier_indices




    
# anorbliy detection: 

# print(round_matrix)
# print(item_matrix)
# print("number of outlier is: ",num_outlier)
# Bidir_all = np.dstack((Bidir_all[:,0],Bidir_all[:,1],Bidir_all[:,2],Bidir_all[:,3],Bidir_all[:,4],Bidir_all[:,5],Bidir_all[:,6],Bidir_all[:,7],Bidir_all[:,8],Bidir_all[:,9],Bidir_all[:,10],Bidir_all[:,11],Bidir_all[:,12],Bidir_all[:,13],Bidir_all[:,14],Bidir_all[:,15],Bidir_all[:,16],Bidir_all[:,17],Bidir_all[:,18],Bidir_all[:,19],Bidir_all[:,20],Bidir_all[:,21],Bidir_all[:,22],Bidir_all[:,23],Bidir_all[:,24],Bidir_all[:,25],Bidir_all[:,26],Bidir_all[:,27],Bidir_all[:,28],Bidir_all[:,29],Bidir_all[:,30],Bidir_all[:,31],Bidir_all[:,32],Bidir_all[:,33],Bidir_all[:,34],Bidir_all[:,35],Bidir_all[:,36],Bidir_all[:,37],Bidir_all[:,38],Bidir_all[:,39],Bidir_all[:,40],Bidir_all[:,41],Bidir_all[:,42],Bidir_all[:,43],Bidir_all[:,44],Bidir_all[:,45],Bidir_all[:,46],Bidir_all[:,47],Bidir_all[:,48],Bidir_all[:,49],Bidir_all[:,50],Bidir_all[:,51],Bidir_all[:,52],Bidir_all[:,53],Bidir_all[:,54],Bidir_all[:,55],Bidir_all[:,56],Bidir_all[:,57],Bidir_all[:,58],Bidir_all[:,59],Bidir_all[:,60],Bidir_all[:,61],Bidir_all[:,62],Bidir_all[:,63],Bidir_all[:,64],Bidir_all[:,65],Bidir_all[:,66],Bidir_all[:,67],Bidir_all[:,68],Bidir_all[:,69],Bidir_all[:,70],Bidir_all[:,71],Bidir_all[:,72],Bidir_all[:,73],Bidir_all[:,74],Bidir_all[:,75],Bidir_all[:,76],Bidir_all[:,77],Bidir_all[:,78],Bidir_all[:,79]))[0]
# plt.plot(Bidir_all[:,15])
# plt.plot(Bidir_all[:,17])


def item_plot(Bidir_all,interested_items, frame_size, color_code, label_des):
    fig, ax = plt.subplots()
    x = [0,1,2,3,4,5,6,7]
    my_xticks = ["64","88","128","256","512","1024","1280","1518"]
    plt.xticks(x, my_xticks)
    
    
    for i in range(len(interested_items)):
        for j in range(len(Bidir_all[:, interested_items[i]])):
            ax.scatter(i,Bidir_all[j, interested_items[i]], s=6, color = color_code)
    plt.title(label_des)
    plt.axis()

# plot_outlier_2d(Bidir_all_new, result_of_interests)

In [3]:
from ipywidgets import widgets
from IPython.display import display
text = widgets.Text()
# display(text)


def handle_submit(sender):
    print(text.value)

def on_button_clicked(b):
    Bidir_all = generate_data()
    
    round_matrix,item_outlier,num_outlier = detetct_outlier_single_item(Bidir_all, 3)
    items_uplink=[5, 15, 25, 35, 45, 55,65,75]
    items_downlink=[7, 17, 27, 37, 47, 57,67,77]

    frame_size = [64,88,128,256,512,1024,1280,1518]
    color_up = "g"
    color_down = "r"
    label_up = "uplink throughput bit/s"
    label_down = "downlink throughput bit/s"
    item_plot(Bidir_all,items_uplink, frame_size, color_up, label_up)
    item_plot(Bidir_all,items_downlink, frame_size, color_down, label_down)
    

    print("abnormal items are: ")
    for i in item_outlier:
        name = index_to_name(i)
        print(name)
    
    array_of_interests = [[5,7],[15,17],[25,27],[35,37],[45,47],[55,57],[65,67],[75,77]]
    plot_indicator = 1
    Bidir_all = generate_data()
    round_matrix,item_matrix,num_outlier = detetct_outlier_single_item(Bidir_all, std_range)
    Bidir_all_new = detect_outlier_multiple(Bidir_all, array_of_interests, plot_indicator)
def operation_proc():
    button = widgets.Button(description = "Start")
    display(button)
    
    text.on_submit(handle_submit)
    button.on_click(on_button_clicked)

In [4]:
operation_proc()

Button(description='Start', style=ButtonStyle())

/Users/tester/Documents/Vz_datamining/Looptest_UDP_BiDir_throughput_bi_M/20180714-043253/unicast_unidirectional_throughput/BidirResults.csv
/Users/tester/Documents/Vz_datamining/Looptest_UDP_BiDir_throughput_bi_M/20180714-130616/unicast_unidirectional_throughput/BidirResults.csv
/Users/tester/Documents/Vz_datamining/Looptest_UDP_BiDir_throughput_bi_M/20180714-035842/unicast_unidirectional_throughput/BidirResults.csv
/Users/tester/Documents/Vz_datamining/Looptest_UDP_BiDir_throughput_bi_M/20180713-225040/unicast_unidirectional_throughput/BidirResults.csv
/Users/tester/Documents/Vz_datamining/Looptest_UDP_BiDir_throughput_bi_M/20180713-221617/unicast_unidirectional_throughput/BidirResults.csv
/Users/tester/Documents/Vz_datamining/Looptest_UDP_BiDir_throughput_bi_M/20180714-021707/unicast_unidirectional_throughput/BidirResults.csv
/Users/tester/Documents/Vz_datamining/Looptest_UDP_BiDir_throughput_bi_M/20180714-112252/unicast_unidirectional_throughput/BidirResults.csv
/Users/tester/Docume

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [5]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="toggle on/off the raw code."></form>''')

